In [1]:
import pandas as pd

In [2]:
xmlfile='data/Wikipedia-20170717213140.xml'
outfile=xmlfile.replace("xml","csv")
data=[]

# Parse file

Loop over lines in the XML file and look for appearances of:
* title
* id
* text

At each step, save the variables into a dictionary.

After reading the file, the dictionary "data" is saved into a "csv" file.


In [3]:
f = open(xmlfile,"r")

state=0
page={}
for line in f.readlines():
    if(state==0):
        # Look for keywork "page" and jump to state=1
        if "page" in line:
            state=1
    elif(state==1):
        # Look for keyword "title" and jump to state=2
        # store "title"
        if "title" in line:
            ll=line.replace("<title>","")
            title=ll.replace("</title>","").strip()
            page['title']=title
            #print(title)
            state=2
    elif(state==2):
        # Look for keywork "id" and jump to state=3
        # store "id"
        if "id" in line:
            ll=line.replace("<id>","")
            aid=ll.replace("</id>","").strip()
            page['id']=aid
            #print(aid)
            state=3
    elif state==3:
        # Look for keywork "<text"
        if ("<text" in line):
            line=line.replace("<text","").strip()
            ll=line.split(">")
            text=ll[1]
            if "</text>" in text:
                # If found </text> then jump to state=5, and store text
                text=text.replace("</text>","")
                state=5
            else:
                state=4
    elif state==4:
        # If found </text> then jump to state=5, and store text
        if "</text>" in line:
            ll=line.replace("</text>","").strip()
            text=text+ll
            state=5
        else:
            text=text+line.strip()
    elif(state==5):
        # "</page>" return to the top (state=0)
        # add current page dictionary into the list.
        if "</page>" in line:
            page['text']=text
            #print(text)
            data.append(page)
            page={}
            state=0
            
f.close()

In [6]:
# Convert list of dictionaries to dataframe
df=pd.DataFrame(data)

df.to_csv(outfile)
df.head()

,id,text,title
0,970284,{{Cat main|Conformation show|Show dog}}{{porta...,Category:Dog shows and showing
1,972913,This is a collection of articles about the hea...,Category:Dog health
2,970251,This is an automatically collected list of art...,Category:Dog organizations
3,729436,This is an automatically accumulated list of a...,Category:Dog sports
4,978163,[[Category:Dogs|Pets]][[Category:Mammals as pe...,Category:Dogs as pets
